# Import and Set

In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import os
import pickle
from torch import nn, Tensor
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from utils import *
from model import TransformerModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_class = 1
stock_symbol = '5871.TW'
end_date = '2024-12-31'

init = True
fp16_training = False
num_epochs = 500
config = {
    'lr': 0.0001,
}

# Init

In [2]:
if num_class == 1:
    with open('../DataLoader/dataloader_1.pk', 'rb') as f:
        data = pickle.load(f)
    trainloader = data['trainloader']
    validloader = data['validloader']
    # dataloader_test = data['testloader']
else:
    with open('../DataLoader/dataloader.pk', 'rb') as f:
        data = pickle.load(f)
    trainloader = data['trainloader']
    validloader = data['validloader']
    # dataloader_test = data['testloader']
with open('../DataLoader/src.pk', 'rb') as f:
    src = pickle.load(f)
    src = src.permute(2, 0, 1)

# Setting and Model

In [3]:
"""
Choose if fp16 and define model
"""
# !pip install accelerate==0.2.0
# Model: ntoken=400, d_model=6, nhead=2, d_hid=128, nlayers=16, num_class=num_class
if fp16_training:
    print('Accelerating')
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device
    model = TransformerModel(num_class=num_class)
else:
    model = TransformerModel(num_class=num_class).to(device)
Model = model.model_type # Model name

"""
Init for models, learning rate, ...
"""
if os.path.exists(f'Temp//{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk'):
    if init:
        print("Init model")
        lr = config['lr']
        last_epoch = 0
        min_val_loss = 10000
        loss_train = []
        loss_valid = []
    else:
        print('Load from last train epoch')
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'rb') as f:
            last_train_info = pickle.load(f)
        lr = last_train_info['lr']
        last_epoch = last_train_info['epoch']
        min_val_loss = last_train_info['min val loss']
        model.load_state_dict(torch.load(f'Temp//{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt'))
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'rb') as f:
            loss_train_val = pickle.load(f)
        loss_train = loss_train_val['train']
        loss_valid = loss_train_val['valid']
else:
    print("Init model")
    lr = config['lr']
    last_epoch = 0
    min_val_loss = 10000.0
    loss_train = []
    loss_valid = []
print(f'Last train epoch: {last_epoch}  '
        f'Last train lr: {lr}   '
        f'Min val loss: {min_val_loss}')

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=len(trainloader)*1, gamma=0.9)        

# Prepare
if fp16_training:
    print('Accelerate Prepare')    
    model, optimizer, trainloader, validloader, scheduler = \
        accelerator.prepare(model, optimizer, trainloader, validloader, scheduler)

/home/jacob/anaconda3/envs/mlntu/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Init model
Last train epoch: 0  Last train lr: 0.0001   Min val loss: 10000.0


# Train

In [4]:
src = src.to(device)
for epoch in range(last_epoch, num_epochs):
    # Training phase
    model.train()
    loss_train_e = 0
    step = 0
    for batch_x, batch_y in tqdm(trainloader):
        step += 1
        if step == len(validloader):
            break
        batch_x = batch_x.permute(2, 0, 1)
        """
        seq_len = src.size(0)
        src_half = src[int(seq_len/2)]
        """
        if not fp16_training:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
        optimizer.zero_grad()
        memory, outputs = model(src=src, tgt=batch_x, train=True)

        # Loss
        loss = criterion(outputs, batch_y*100)
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()
        scheduler.step()
        loss_train_e += loss.item()
        
    loss_train_e /= len(trainloader)
    loss_train.append(loss_train_e)
    
    loss_valid_e = 0
    with torch.no_grad():
        model.eval()
        step = 0
        for batch_x_val, batch_y_val in tqdm(validloader):
            step += 1
            if step == len(validloader):
                break
            batch_x_val = batch_x_val.permute(2, 0, 1)
            if not fp16_training:
                # Use src as trained data
                batch_x_val = batch_x_val.to(device)
                batch_y_val = batch_y_val.to(device)
            memory, outputs_val = model(src=src, tgt=batch_x_val, train=False, memory=memory)
            loss = criterion(outputs_val, batch_y_val)
            loss_valid_e += loss.item()
        loss_valid_e /= len(validloader)
        loss_valid.append(loss_valid_e)
            
        torch.save(model.state_dict(), f'Temp/{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt')
        if loss_valid_e < min_val_loss:
            min_val_loss = loss_valid_e
            print(f'New best model found in epoch {epoch} with val loss: {min_val_loss}')
            torch.save(model.state_dict(), f'Model_Result/{Model}_class{num_class}_{stock_symbol}_best_model.pt')            
        if epoch % 50 == 0:
            pass
            # torch.save(model, f'ConformerResult/Conformerr_{stock_symbol}_checkpoint_{epoch}.pt')
            
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'wb') as f:
        pickle.dump({'train': loss_train, 'valid': loss_valid}, f)
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'wb') as f:
        pickle.dump({'min val loss': min_val_loss, 'epoch': epoch, 'lr': optimizer.param_groups[0]['lr']}, f)
        
    # Print statistics
    print(f'Epoch [{epoch}/{num_epochs}]',
        f'Training Loss: {loss_train_e:.10f}',
        f'Valid Loss: {loss_valid_e:.10f}')

 93%|█████████▎| 14/15 [00:01<00:00,  8.58it/s]


New best model found in epoch 0 with val loss: 0.16939438184102376
Epoch [0/500] Training Loss: 1.1974412285 Valid Loss: 0.1693943818

















 93%|█████████▎| 14/15 [00:01<00:00,  8.23it/s]



Epoch [1/500] Training Loss: 0.9189246153 Valid Loss: 0.2153822452


New best model found in epoch 2 with val loss: 0.11347600320974986
Epoch [2/500] Training Loss: 0.7906529821 Valid Loss: 0.1134760032


...























































































































































































































































































































































































































 93%|█████████▎| 14/15 [00:01<00:00,  7.99it/s]

















Epoch [11/500] Training Loss: 0.6968008917 Valid Loss: 0.0189482205


New best model found in epoch 12 with val loss: 0.00987645573914051
Epoch [12/500] Training Loss: 0.6152949970 Valid Loss: 0.0098764557






























































































































































































































































































































































































































































































































 93%|█████████▎| 14/15 [00:01<00:00,  8.15it/s]A




















New best model found in epoch 13 with val loss: 0.00789072442178925
Epoch [13/500] Training Loss: 0.6281702272 Valid Loss: 0.0078907244


In [ ]:
outputs.shape, batch_y.shape

(torch.Size([32, 1]), torch.Size([32, 1]))

# Validate Model

In [ ]:
def load_model():
    import torch
    model = torch.load(f'ConformerResult/Conformer_{stock_symbol}_best_model.pt')
    return model
model = load_model()

In [ ]:

import gc
def test():
    dataloader = dataloader_test

    model.eval()
    s_pred = []
    s_true = []
    for x, y in tqdm(dataloader):
        y_pred = model(x)
        s_pred.append(y_pred.detach())
        s_true.append(y)
    y_pred_tensor = torch.concat(s_pred)
    y_test_tensor = torch.concat(s_true)
    accuracy = (torch.sign(y_pred_tensor) == torch.sign(y_test_tensor)).sum() / len(y_test_tensor)
    return y_pred_tensor, accuracy

y_pred, acc = test()
print(acc)

In [ ]:
# Derive y_pred and y_train_pred of shape(N, 2) and numpy type

y_pred_numpy = y_pred.cpu().numpy()

# predict with train set
y_train_pred = model(torch.tensor(X[-100:], dtype = torch.float32))
y_train_numpy = y_train_pred.detach().cpu().numpy()


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Scaling
prediction = pd.DataFrame(y_pred_numpy)
scaler = StandardScaler()
scaler.fit(y_train_numpy)
prediction = pd.DataFrame(scaler.transform(prediction))

# Get the predicted price of O and C and Prediction merge with complete data
prediction.columns = ['pred_do_1', 'pred_dc_1']
prediction['Date'] = date

true_and_pred = pd.merge(df.reset_index(), prediction, on = 'Date', how = 'left')
true_and_pred['pred_o'] = (true_and_pred['Open'] * (1 + true_and_pred['pred_do_1'])).shift(1)
true_and_pred['pred_c'] = (true_and_pred['Close'] * (1 + true_and_pred['pred_dc_1'])).shift(1)
true_and_pred['pred_oc'] = true_and_pred['pred_c'] - true_and_pred['pred_o']
true_and_pred['true_oc'] = true_and_pred['Close'] - true_and_pred['Open']

# Backtest
asset_list = []
df_backtest = true_and_pred[['Open', 'Close', 'true_oc', 'pred_oc']].dropna()
asset = 1
for index, (o, c, true, pred) in df_backtest.iterrows():
    if pred > 0:
        returns = true/o
        asset *= (1 + returns)
    asset_list.append(asset)

print(asset)
plt.plot(asset_list, label = 'resnet')
plt.plot(df_backtest.reset_index()['Close']/df_backtest['Close'].iloc[0], label = 'buy hold')
plt.legend()
plt.savefig('/ConformerResult/test_backtest.jpg')
# plt.show()